In [1]:
from tsai.models.utils import *
from tsai.basics import *
from tsai.inference import load_learner
from tsai.all import *
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

In [2]:
import numpy as np
import pandas as pd
import csv
from tabulate import tabulate
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.utils import to_categorical


2024-03-29 15:41:24.251620: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 15:41:24.251677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 15:41:24.252960: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-29 15:41:24.259128: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-29 15:41:24.955160: W tensorflow/compiler/tf2

In [3]:
def train_model_lstm(x_train, y_train, x_valid, y_valid):
    X, y, splits = combine_split_data([x_train, x_valid], [y_train, y_valid])
    tfms  = [None, TSClassification()] # TSClassification == Categorize
    batch_tfms = TSStandardize()
    dls = get_ts_dls(X, y, splits=splits, tfms=tfms, batch_tfms=batch_tfms, bs=[64, 32])
    #dls.dataset
    model = build_ts_model(LSTM_FCNPlus, dls=dls)
    learn = Learner(dls, model, metrics=accuracy)
    learn.fit_one_cycle(10, lr_max=1e-2)
    return learn

def train_model_cnn(x_train, y_train, x_valid, y_valid):
    X, y, splits = combine_split_data([x_train, x_valid], [y_train, y_valid])
    tfms  = [None, TSClassification()] # TSClassification == Categorize
    batch_tfms = TSStandardize()
    dls = get_ts_dls(X, y, splits=splits, tfms=tfms, batch_tfms=batch_tfms, bs=[64, 32])
    #dls.dataset
    model = build_ts_model(InceptionTimePlus, dls=dls)
    learn = Learner(dls, model, metrics=accuracy)
    learn.fit_one_cycle(10, lr_max=1e-2)
    return learn

def train_model_tst(x_train, y_train, x_valid, y_valid):
    X, y, splits = combine_split_data([x_train, x_valid], [y_train, y_valid])
    tfms  = [None, TSClassification()] # TSClassification == Categorize
    batch_tfms = TSStandardize()
    dls = get_ts_dls(X, y, splits=splits, tfms=tfms, batch_tfms=batch_tfms, bs=[64, 32])
    #dls.dataset
    model = build_ts_model(TSTPlus, dls=dls)
    learn = Learner(dls, model, metrics=accuracy)
    learn.fit_one_cycle(10, lr_max=1e-2)
    return learn

def run_model(x_test, y_test, learn):
    probas, target, preds = learn.get_X_preds(x_test, y_test)
    preds_labels = np.argmax(preds, axis=1)
    target_labels = np.argmax(target, axis=-1) # undo one-hot encoding
    return preds_labels, target_labels

In [4]:
x_train_new = np.load('x_train_diff.npy')
y_train_new = np.load('y_train_diff.npy')
print(x_train_new.shape, y_train_new.shape)

(13800, 128, 1) (13800, 2)


In [5]:
input_dir = '../eeg_dataset_1/'
x_validation = np.load('x_valid.npy')
y_validation = np.load('y_valid.npy')
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')
# x_validation = x_validation.astype(float)
# x_test = x_test.astype(float)
# x_validation = x_validation[:, :4096,:]
# x_test = x_test[:, :4096,:]
print(x_validation.shape, y_validation.shape)
print(x_test.shape, y_test.shape)

(2300, 128, 1) (2300, 2)
(2300, 128, 1) (2300, 2)


In [6]:
# input_dir = '../eeg_dataset_1/'
# x_validation = np.load(input_dir +'x_valid.npy')
# y_validation = np.load(input_dir +'y_valid.npy')
# x_test = np.load(input_dir +'x_test.npy')
# y_test = np.load(input_dir +'y_test.npy')
# x_validation = x_validation.astype(float)
# x_test = x_test.astype(float)
# x_validation = x_validation[:, :4096,:]
# x_test = x_test[:, :4096,:]
# print(x_validation.shape, y_validation.shape)
# print(x_test.shape, y_test.shape)

In [7]:
Z=1.96
acc_list_mean = []
for shuffle_id in range(10):
    x_train_shuffled, y_train_shuffled = shuffle(x_train_new, y_train_new)
    learn = train_model_lstm(x_train_shuffled, y_train_shuffled, x_validation, y_validation)
    preds_labels, target_labels = run_model(x_test, y_test, learn)
    acc_list_mean.append(accuracy_score(target_labels, preds_labels))

mean_accuracy = np.mean(acc_list_mean)
std_dev = np.std(acc_list_mean, ddof=1)  # ddof=1 for sample standard deviation
std_error = std_dev / np.sqrt(len(acc_list_mean))
interval_range = Z*std_error  
print(mean_accuracy)
print(interval_range)

epoch,train_loss,valid_loss,accuracy,time
0,0.347257,0.190533,0.922391,00:01
1,0.348997,0.172080,0.946957,00:01
2,0.347553,0.174056,0.941087,00:01
3,0.312253,0.165667,0.937826,00:01
4,0.314015,0.123117,0.955000,00:01
5,0.300705,0.128113,0.950652,00:01
6,0.283943,0.152925,0.951304,00:01
7,0.283514,0.135191,0.954130,00:01
8,0.282443,0.130860,0.955652,00:01
9,0.278654,0.131001,0.958261,00:01


epoch,train_loss,valid_loss,accuracy,time
0,0.342887,0.165446,0.947174,00:01
1,0.348902,0.214965,0.916087,00:01
2,0.343025,0.142129,0.951957,00:01
3,0.335505,0.132595,0.946957,00:01
4,0.304984,0.124947,0.951739,00:01
5,0.305309,0.157163,0.945217,00:01
6,0.285093,0.131158,0.948261,00:01
7,0.278743,0.120874,0.957391,00:01
8,0.283148,0.133270,0.955217,00:01
9,0.280032,0.119438,0.960217,00:01


epoch,train_loss,valid_loss,accuracy,time
0,0.338411,0.167422,0.938696,00:01
1,0.343790,0.159867,0.944783,00:01
2,0.332636,0.151288,0.947826,00:01
3,0.317719,0.141673,0.950435,00:01
4,0.328083,0.157901,0.946087,00:01
5,0.316093,0.140688,0.946957,00:01
6,0.299362,0.129006,0.951304,00:01
7,0.293446,0.119732,0.953261,00:01
8,0.274057,0.136498,0.953261,00:01
9,0.275671,0.122340,0.955652,00:01


epoch,train_loss,valid_loss,accuracy,time
0,0.334446,0.147682,0.947609,00:01
1,0.341151,0.216596,0.925217,00:01
2,0.334494,0.168513,0.929130,00:01
3,0.337897,0.147188,0.949130,00:01
4,0.316691,0.127815,0.951522,00:01
5,0.314445,0.175988,0.940435,00:01
6,0.296212,0.142585,0.950435,00:01
7,0.296432,0.131120,0.956522,00:01
8,0.282591,0.124461,0.960217,00:01
9,0.275697,0.126548,0.958261,00:01


epoch,train_loss,valid_loss,accuracy,time
0,0.324978,0.171453,0.944348,00:01
1,0.342591,0.141829,0.946739,00:01
2,0.337351,0.151701,0.945000,00:01
3,0.331236,0.161952,0.942826,00:01
4,0.316343,0.133923,0.947391,00:01
5,0.298850,0.150296,0.945435,00:01
6,0.299519,0.125641,0.951304,00:01
7,0.289078,0.127304,0.957609,00:01
8,0.275532,0.119020,0.958696,00:01
9,0.279404,0.103351,0.963043,00:01


...

epoch,train_loss,valid_loss,accuracy,time
0,0.352754,0.198755,0.926087,00:01
1,0.347091,0.172187,0.945652,00:01
2,0.323526,0.184433,0.933261,00:01
3,0.325229,0.159274,0.945217,00:01
4,0.311880,0.145509,0.953478,00:01
5,0.302617,0.151090,0.948043,00:01
6,0.291042,0.170082,0.944348,00:01
7,0.278176,0.133822,0.952174,00:01
8,0.282279,0.134332,0.954783,00:01
9,0.267478,0.131788,0.956522,00:01


epoch,train_loss,valid_loss,accuracy,time
0,0.342037,0.145037,0.946957,00:01
1,0.335352,0.181371,0.941087,00:01
2,0.334647,0.200502,0.925652,00:01
3,0.315274,0.124561,0.952826,00:01
4,0.318603,0.140190,0.947391,00:01
5,0.310703,0.189598,0.941957,00:01
6,0.294525,0.124129,0.953913,00:01
7,0.290023,0.112345,0.958478,00:01
8,0.273916,0.125969,0.957391,00:01
9,0.272952,0.106004,0.960435,00:01


epoch,train_loss,valid_loss,accuracy,time
0,0.329472,0.159334,0.949783,00:01
1,0.368953,0.246701,0.930000,00:01
2,0.330665,0.158966,0.944783,00:01
3,0.332397,0.152999,0.941087,00:01
4,0.311739,0.145316,0.946304,00:01
5,0.303938,0.186132,0.947826,00:01
6,0.291328,0.141776,0.950870,00:01
7,0.290810,0.119614,0.953043,00:01
8,0.284040,0.154027,0.949565,00:01
9,0.283962,0.150874,0.949565,00:01


epoch,train_loss,valid_loss,accuracy,time
0,0.345631,0.159504,0.943043,00:01
1,0.349729,0.164820,0.946957,00:01
2,0.357361,0.174229,0.936522,00:01
3,0.329686,0.146713,0.942609,00:01
4,0.309533,0.117911,0.951957,00:01
5,0.308392,0.152025,0.950000,00:01
6,0.296903,0.140552,0.946087,00:01
7,0.290728,0.129099,0.951522,00:01
8,0.287150,0.131510,0.952391,00:01
9,0.278490,0.119120,0.956304,00:01


0.9695217391304347
0.0020144096740245363


In [8]:
Z=1.96
acc_list_mean = []
for shuffle_id in range(10):
    x_train_shuffled, y_train_shuffled = shuffle(x_train_new, y_train_new)
    learn = train_model_cnn(x_train_shuffled, y_train_shuffled, x_validation, y_validation)
    preds_labels, target_labels = run_model(x_test, y_test, learn)
    acc_list_mean.append(accuracy_score(target_labels, preds_labels))

mean_accuracy = np.mean(acc_list_mean)
std_dev = np.std(acc_list_mean, ddof=1)  # ddof=1 for sample standard deviation
std_error = std_dev / np.sqrt(len(acc_list_mean))
interval_range = Z*std_error  
print(mean_accuracy)
print(interval_range)

epoch,train_loss,valid_loss,accuracy,time
0,0.361008,0.185447,0.928696,00:04
1,0.370215,0.541985,0.831087,00:04
2,0.342855,0.258409,0.881739,00:04
3,0.315982,0.159500,0.943478,00:04
4,0.317388,0.137933,0.945652,00:04
5,0.304012,0.138673,0.947391,00:04
6,0.295656,0.178672,0.937826,00:04
7,0.298732,0.146607,0.947391,00:04
8,0.281025,0.156287,0.946957,00:04
9,0.286445,0.145589,0.948913,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.367758,0.184832,0.946087,00:04
1,0.361658,0.183136,0.933478,00:04
2,0.328276,0.178977,0.928696,00:04
3,0.321804,0.167604,0.936739,00:04
4,0.317071,0.171562,0.933043,00:04
5,0.312713,0.159200,0.940217,00:04
6,0.301365,0.159371,0.937609,00:04
7,0.284391,0.138662,0.945000,00:04
8,0.284341,0.154972,0.943913,00:04
9,0.283885,0.137625,0.947174,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.347249,0.165456,0.948478,00:04
1,0.367436,0.209730,0.937826,00:04
2,0.336813,0.209657,0.911739,00:04
3,0.325216,0.174183,0.934783,00:04
4,0.323717,0.129871,0.938913,00:04
5,0.317852,0.150926,0.943043,00:04
6,0.295929,0.121684,0.950870,00:04
7,0.294656,0.126292,0.951739,00:04
8,0.281850,0.146913,0.948913,00:04
9,0.283276,0.129166,0.953478,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.340893,0.192096,0.939783,00:04
1,0.346577,0.176317,0.942174,00:04
2,0.331181,0.187422,0.937391,00:04
3,0.317693,0.210242,0.932609,00:04
4,0.320815,0.146330,0.945652,00:04
5,0.298724,0.151551,0.946304,00:04
6,0.288368,0.146341,0.948261,00:04
7,0.279934,0.162237,0.946087,00:04
8,0.298172,0.158978,0.946522,00:04
9,0.290457,0.161916,0.946087,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.345345,0.168436,0.946739,00:04
1,0.338845,0.246075,0.910652,00:03
2,0.350568,0.219455,0.917826,00:03
3,0.317893,0.172283,0.937609,00:04
4,0.322362,0.170669,0.935652,00:04
5,0.315168,0.156961,0.944130,00:04
6,0.300184,0.180832,0.939348,00:04
7,0.292578,0.158478,0.943043,00:04
8,0.284339,0.170269,0.942391,00:04
9,0.283357,0.169302,0.942609,00:04


...

epoch,train_loss,valid_loss,accuracy,time
0,0.350056,0.181654,0.925435,00:04
1,0.362330,0.207377,0.907609,00:04
2,0.330988,0.146242,0.938478,00:04
3,0.320874,0.159031,0.942826,00:04
4,0.318806,0.176387,0.944565,00:04
5,0.314178,0.127816,0.946522,00:04
6,0.301768,0.124246,0.947391,00:04
7,0.287898,0.138404,0.945000,00:04
8,0.288723,0.124763,0.948913,00:03
9,0.285409,0.110966,0.953043,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.353333,0.187979,0.936087,00:04
1,0.346534,0.151388,0.945217,00:04
2,0.341143,0.186059,0.935652,00:04
3,0.342153,0.167435,0.933261,00:04
4,0.315391,0.134641,0.942609,00:04
5,0.302857,0.180901,0.939783,00:04
6,0.295868,0.131135,0.948696,00:04
7,0.292969,0.128467,0.948913,00:04
8,0.276262,0.134292,0.951522,00:04
9,0.290125,0.144576,0.950217,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.341715,0.193011,0.931739,00:04
1,0.338951,0.160919,0.935435,00:04
2,0.355931,0.201579,0.932609,00:04
3,0.327680,0.208673,0.927174,00:04
4,0.308513,0.163342,0.936957,00:04
5,0.304073,0.159565,0.940870,00:04
6,0.302402,0.144249,0.944348,00:04
7,0.292344,0.151838,0.938043,00:04
8,0.281402,0.158843,0.940870,00:04
9,0.290433,0.132859,0.946522,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.346433,0.144417,0.946522,00:04
1,0.324967,0.155123,0.940435,00:04
2,0.326490,0.166095,0.943696,00:04
3,0.328376,0.183759,0.927174,00:04
4,0.315005,0.137706,0.949783,00:04
5,0.307701,0.141022,0.948261,00:04
6,0.301856,0.145848,0.944348,00:04
7,0.291608,0.166811,0.939565,00:04
8,0.273244,0.150960,0.948478,00:04
9,0.284216,0.123965,0.952609,00:04


0.9677826086956521
0.0017017419692446506


In [9]:
Z=1.96
acc_list_mean = []
for shuffle_id in range(10):
    x_train_shuffled, y_train_shuffled = shuffle(x_train_new, y_train_new)
    learn = train_model_tst(x_train_shuffled, y_train_shuffled, x_validation, y_validation)
    preds_labels, target_labels = run_model(x_test, y_test, learn)
    acc_list_mean.append(accuracy_score(target_labels, preds_labels))

mean_accuracy = np.mean(acc_list_mean)
std_dev = np.std(acc_list_mean, ddof=1)  # ddof=1 for sample standard deviation
std_error = std_dev / np.sqrt(len(acc_list_mean))
interval_range = Z*std_error  
print(mean_accuracy)
print(interval_range)

epoch,train_loss,valid_loss,accuracy,time
0,0.394791,0.299981,0.932826,00:04
1,0.387344,0.265341,0.932391,00:04
2,0.402754,0.294487,0.912174,00:04
3,0.385116,0.254397,0.922826,00:04
4,0.367244,0.357612,0.856522,00:04
5,0.369199,0.201883,0.930652,00:04
6,0.346437,0.199612,0.933261,00:04
7,0.335939,0.194689,0.929348,00:04
8,0.341576,0.176020,0.945435,00:04
9,0.325031,0.180032,0.943478,00:03


epoch,train_loss,valid_loss,accuracy,time
0,0.409170,0.276813,0.927391,00:03
1,0.414676,0.266965,0.923478,00:03
2,0.388171,0.207188,0.934348,00:04
3,0.374287,0.201575,0.932609,00:04
4,0.377104,0.319712,0.872391,00:04
5,0.359087,0.197780,0.941087,00:04
6,0.352688,0.193162,0.946522,00:03
7,0.330177,0.197637,0.940217,00:03
8,0.337012,0.168432,0.947174,00:04
9,0.329520,0.199300,0.945217,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.410036,0.263949,0.934783,00:04
1,0.400816,0.235384,0.929783,00:04
2,0.377814,0.232763,0.925652,00:04
3,0.369800,0.245903,0.911087,00:04
4,0.374546,0.229891,0.926957,00:03
5,0.364233,0.203199,0.934565,00:03
6,0.348921,0.186675,0.930435,00:04
7,0.341037,0.193222,0.944783,00:04
8,0.335944,0.200704,0.933043,00:04
9,0.334726,0.180303,0.940000,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.416948,0.214357,0.937391,00:03
1,0.438106,0.318685,0.894565,00:03
2,0.385169,0.212192,0.929130,00:03
3,0.369224,0.214231,0.929348,00:03
4,0.361003,0.196102,0.937391,00:03
5,0.355140,0.200831,0.926087,00:03
6,0.366521,0.225917,0.919565,00:03
7,0.343657,0.210226,0.928913,00:03
8,0.330218,0.227214,0.929565,00:04
9,0.334870,0.206809,0.932609,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.422537,0.239786,0.939783,00:04
1,0.376017,0.207385,0.933913,00:04
2,0.378667,0.264127,0.927826,00:03
3,0.377127,0.196838,0.933478,00:03
4,0.337951,0.180592,0.946087,00:03
5,0.340701,0.191900,0.932609,00:04
6,0.322545,0.161578,0.943913,00:04
7,0.325125,0.167707,0.947826,00:04
8,0.316843,0.181798,0.938696,00:04
9,0.323883,0.189145,0.937826,00:04


...

epoch,train_loss,valid_loss,accuracy,time
0,0.403859,0.312069,0.931087,00:03
1,0.401432,0.551655,0.853261,00:03
2,0.393889,0.258905,0.924348,00:04
3,0.377971,0.331567,0.910435,00:04
4,0.374455,0.215145,0.929348,00:04
5,0.351981,0.203189,0.935652,00:04
6,0.358708,0.217297,0.924130,00:04
7,0.345655,0.201393,0.939565,00:04
8,0.342101,0.210050,0.923043,00:04
9,0.323180,0.201797,0.936522,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.396527,0.250297,0.937609,00:04
1,0.405282,0.275255,0.895652,00:04
2,0.385542,0.194024,0.943478,00:04
3,0.378861,0.238166,0.921087,00:04
4,0.365690,0.201631,0.927391,00:04
5,0.354743,0.179203,0.939565,00:03
6,0.350949,0.184762,0.942391,00:03
7,0.336108,0.169455,0.945217,00:03
8,0.339268,0.162546,0.945652,00:03
9,0.329817,0.166163,0.943913,00:03


epoch,train_loss,valid_loss,accuracy,time
0,0.415063,0.275745,0.938261,00:03
1,0.396686,0.282643,0.909130,00:03
2,0.379029,0.199886,0.936739,00:04
3,0.373466,0.238094,0.914565,00:04
4,0.363018,0.206190,0.933261,00:04
5,0.345184,0.191194,0.935652,00:04
6,0.342249,0.257094,0.892609,00:04
7,0.329886,0.176734,0.929348,00:04
8,0.320863,0.173403,0.945000,00:03
9,0.323394,0.167763,0.944130,00:04


epoch,train_loss,valid_loss,accuracy,time
0,0.416914,0.275693,0.932609,00:04
1,0.404641,0.348499,0.900870,00:04
2,0.363314,0.194634,0.929130,00:03
3,0.362438,0.214545,0.923043,00:03
4,0.354887,0.190186,0.943696,00:03
5,0.344155,0.207866,0.915870,00:03
6,0.335870,0.182535,0.935652,00:03
7,0.336368,0.192277,0.930435,00:04
8,0.315732,0.162135,0.949565,00:04
9,0.313445,0.171380,0.946087,00:03


0.9580869565217391
0.003662475648014027


In [8]:
learn_lstm = train_model_lstm(x_train_new, y_train_new, x_validation, y_validation)
preds_labels_lstm, target_labels_lstm = run_model(x_test, y_test, learn_lstm)
acc_lstm = accuracy_score(target_labels_lstm, preds_labels_lstm)

learn_cnn = train_model_cnn(x_train_new, y_train_new, x_validation, y_validation)
preds_labels_cnn, target_labels_cnn = run_model(x_test, y_test, learn_cnn)
acc_cnn = accuracy_score(target_labels_cnn, preds_labels_cnn)

learn_tst = train_model_tst(x_train_new, y_train_new, x_validation, y_validation)
preds_labels_tst, target_labels_tst = run_model(x_test, y_test, learn_tst)
acc_tst = accuracy_score(target_labels_tst, preds_labels_tst)

epoch,train_loss,valid_loss,accuracy,time
0,0.608415,0.594107,0.765000,00:00
1,0.477458,0.198141,0.970000,00:00
2,0.410459,0.158896,0.970000,00:00
3,0.368046,0.286263,0.955000,00:00
4,0.345462,0.265829,0.960000,00:00
5,0.323179,0.245110,0.960000,00:00
6,0.316508,0.261026,0.940000,00:00
7,0.303703,0.350701,0.940000,00:00
8,0.294466,0.376594,0.930000,00:00
9,0.290368,0.368231,0.930000,00:00


epoch,train_loss,valid_loss,accuracy,time
0,0.579430,0.663210,0.850000,00:00
1,0.482931,0.638918,0.830000,00:00
2,0.411291,0.248830,0.960000,00:00
3,0.373573,0.301757,0.920000,00:00
4,0.347958,0.389345,0.890000,00:00
5,0.332384,0.568607,0.890000,00:00
6,0.316100,0.677755,0.880000,00:00
7,0.305764,0.721702,0.880000,00:00
8,0.298760,0.746996,0.880000,00:00
9,0.291326,0.769394,0.880000,00:00


epoch,train_loss,valid_loss,accuracy,time
0,0.673313,1.129208,0.670000,00:00
1,0.531355,1.016886,0.795000,00:00
2,0.493001,3.545579,0.740000,00:00
3,0.467897,5.755565,0.720000,00:00
4,0.452342,0.350376,0.880000,00:00
5,0.435339,0.347084,0.880000,00:00
6,0.419137,0.360980,0.880000,00:00
7,0.407665,0.363724,0.880000,00:00
8,0.396609,0.364463,0.880000,00:00
9,0.390761,0.366379,0.880000,00:00


In [9]:
print('LSTM_FCN: ', acc_lstm)
print('InceptionTime: ', acc_cnn)
print('TST: ', acc_tst)

LSTM_FCN:  0.93
InceptionTime:  0.89
TST:  0.88
